## Load the Dataset
MatchZoo expect a list of *Quintuple* as training data. The corresponded columns are `(text_left_id, text_right_id, text_left, text_right, label)`. For Information Retrieval task, `text_left` is referred as `query`, and `text_right` is document.

For the test case, MatchZoo expect a list of *Quadruple* (we do not need labels) as input.

MatchZoo expect a list of *Quintuple* as training data:

```python
train = [('qid0', 'did0', 'query 0', 'document 0', 'label 0'),
         ('qid0', 'did1', 'query 0', 'document 1', 'label 1'),
          ...,
         ('qid1', 'did2', 'query 1', 'document 2', 'label 3')]
```

The corresponded columns are `(text_left_id, text_right_id, text_left, text_right, label)`. For Information Retrieval task, *text_left* is referred as *query*, and *text_right* is document.

For the test case, MatchZoo expect a list of *Quadruple* (we do not need labels) as input:

```python
test = [('qid9', 'did5', 'query 9', 'document 5'),
         ...,
        ('qid2', 'did7', 'query 2', 'document 7')]
```

In [4]:
def read_data(path, stage):
    def scan_file():
        with open(path) as in_file:
            next(in_file)  # skip header
            for l in in_file:
                yield l.strip().split('\t')
    if stage == 'train':
        return [(qid, did, q, d, label) for qid, did, q, d, label in scan_file()]
    elif stage == 'predict':
        return [(elem[0], elem[1], elem[2], elem[3]) for elem in scan_file()]

train = read_data('data/matchzoo_input.txt', stage='train')
predict  = read_data('data/matchzoo_predict.txt', stage='predict')
rank = read_data('data/matchzoo_rank.txt', stage='predict')

In [5]:
print(train[0])
print(predict[0])
print(rank[0])

('350', 'FT934-11789', 'Health and Computer Terminals', "11 18 20,000 29 70 93 931029 _an a a a a a a a a a a action action after against against agency agree also although an an and and and and and and and and and and and anything appeal are arms as as as as as as as ascribe at at at authentic award be be because because been being being bernard between books both britain brought but by by by by by by care case case case case cast causal cause charter claim claim claim clerical colleague come comp company company computer computer concept condition condition condition conditions conditions confidence confuse considering continue correspondent costs could could country court court court court court court court criticise damages damages damages describe disappointed dismiss disorder dj2dcad8ft doubt down due ec editor elbow emergence emotional employ employee employer even exist expert factor felt financial first for for for for former ft future future gbz go greatest had had hand he he

## Preprocessing

In [6]:
from matchzoo import preprocessor
dssm_preprocessor = preprocessor.DSSMPreprocessor()
datapack_train = dssm_preprocessor.fit_transform(train, stage='train')

Using TensorFlow backend.


ModuleNotFoundError: No module named 'tensorflow'

In [173]:
type(datapack_train)

matchzoo.datapack.DataPack

In [174]:
# pre-processed records including index and processed text to store `text_left` and `id_left`
datapack_train.left.head()

,text_left,length_left
id_left,,
350,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",11196


In [175]:
# pre-processed records including index and processed text to store `text_right` and `id_right`
datapack_train.right.head()

,text_right,length_right
id_right,,
FT934-11789,"[0.0, 2.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",11196
LA091090-0108,"[0.0, 0.0, 2.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",11196
LA120789-0021,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",11196
LA031990-0076,"[0.0, 2.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, ...",11196
FT921-12910,"[0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",11196


In [176]:
# pre-processed records including index and index mapping `id_left` and `id_right`
datapack_train.relation.head()

,id_left,id_right,label
0,350,FT934-11789,1
1,350,LA091090-0108,1
2,350,LA120789-0021,1
3,350,LA031990-0076,1
4,350,FT921-12910,1


In [177]:
# other information stored during the pre-processing process
datapack_train.context.keys()

dict_keys(['term_index', 'input_shapes'])

In [178]:
# vocabulary size
len(datapack_train.context['term_index'])

11195

In [179]:
# since DSSM models' input shapes are dynamic
# (depend on the generated tri-letters)
# so we have to calculate shapes during the pre-processing process
datapack_train.context['input_shapes']

[(11196,), (11196,)]

## Data Generation

In [180]:
from matchzoo import generators
from matchzoo import tasks
generator_train = generators.PointGenerator(
    inputs=datapack_train, task=tasks.Ranking(), batch_size=64, stage='train')
#generator_predict = generators.PointGenerator(
#   inputs=datapack_predict, task=tasks.Ranking(), batch_size=64, stage='predict')

## Training

In [181]:
from matchzoo import models, load_model
from matchzoo import losses
from matchzoo import tasks
from matchzoo import metrics
dssm_model = models.DSSMModel()

In [182]:
# handle dynamic input shapes of DSSM
input_shapes = datapack_train.context['input_shapes']
dssm_model.params['input_shapes'] = input_shapes

In [183]:
dssm_model.params['task'] = tasks.Ranking()
dssm_model.params['task'].metrics = ['mae', 'map']

In [184]:
dssm_model.guess_and_fill_missing_params()
print(dssm_model.params)

name                          DSSMModel
model_class                   <class 'matchzoo.models.dssm_model.DSSMModel'>
input_shapes                  [(11196,), (11196,)]
task                          <matchzoo.tasks.ranking.Ranking object at 0x1473e1ef0>
optimizer                     adam
w_initializer                 glorot_normal
b_initializer                 zeros
dim_fan_out                   128
dim_hidden                    300
activation_hidden             tanh
num_hidden_layers             2


In [185]:
dssm_model.build()
dssm_model.compile()
dssm_model.fit_generator(generator_train, steps_per_epoch=20, epochs=10)

Epoch 1/10
20/20 [==============================] - 2s 101ms/step - loss: 0.2038 - mean_absolute_error: 0.3021
Epoch 2/10
20/20 [==============================] - 1s 63ms/step - loss: 0.0110 - mean_absolute_error: 0.0670
Epoch 3/10
20/20 [==============================] - 1s 65ms/step - loss: 0.0041 - mean_absolute_error: 0.0316
Epoch 4/10
20/20 [==============================] - 1s 63ms/step - loss: 0.0029 - mean_absolute_error: 0.0276
Epoch 5/10
20/20 [==============================] - 1s 60ms/step - loss: 0.0019 - mean_absolute_error: 0.0177
Epoch 6/10
20/20 [==============================] - 1s 68ms/step - loss: 0.0012 - mean_absolute_error: 0.0136
Epoch 7/10
20/20 [==============================] - 1s 61ms/step - loss: 8.7748e-04 - mean_absolute_error: 0.0145
Epoch 8/10
20/20 [==============================] - 1s 67ms/step - loss: 0.0011 - mean_absolute_error: 0.0171
Epoch 9/10
20/20 [==============================] - 1s 63ms/step - loss: 7.9090e-04 - mean_absolute_error: 0.0201
E

In [189]:
X, Y = generator_train[0]
dssm_model.evaluate(X, Y)

64/64 [==============================] - 0s 213us/step


{'loss': 0.0005072275525890291,
 'mean_absolute_error': 0.008711242116987705,
 'mean_average_precision(0)': 1.0}

## Prediction Function

In [ ]:
global toprank, kthrank, 
datapack_rank = dssm_preprocessor.fit_transform(rank, stage='predict')
    generator_rank = generators.PointGenerator(
        inputs=datapack_predict, task=tasks.Ranking(), batch_size=len(doc_text), stage='predict')
X_rank, _ = generator_predict[0]
k = 10
ranking = dssm_model.predict(X_rank)
rank_list = [r[0] for r in ranking]
rank_list.sort(reverse=True)
toprank = rank_list[0]
kthrank = rank_list[k]

In [202]:
import numpy as np

def predict_proba(doc_text):
    predict_data = list()
    count = 1
    did_list = list()
    for doc in doc_text:
        did_list.append(did + "_PRED_"+str(count))
        predict_data.append((qid, did + "_PRED_"+ str(count), query, doc))
        count += 1
        
    datapack_predict = dssm_preprocessor.fit_transform(predict_data, stage='predict')
    generator_predict = generators.PointGenerator(
        inputs=datapack_predict, task=tasks.Ranking(), batch_size=len(doc_text), stage='predict')
    X_predict, _ = generator_predict[0]
    
    pred = dssm_model.predict(X_predict)
    pred_list = [p[0] for p in pred]
    pdoclist = list(zip(did_list, pred_list))
#     pdoclist.sort(key=lambda x: x[1], reverse = True)
    
#     k = len(doc_text) // 10
#     topscore = pdoclist[0][1]
#     kscore = pdoclist[k][1]
    
    newdoclist = list()
    for i in range(len(pdoclist)):
        if pdoclist[i][1] > kthscore:
            newdoclist.append((pdoclist[i][0], 1))
        else:
            newdoclist.append((pdoclist[i][0],0))
            
#     newdoclist.sort(key=lambda x:x[0])
    prob = [(1 - elem[1], elem[1]) for elem in newdoclist]
#     print(len(prob))   
#     print(prob)
    return np.array(prob)


## Lime Initialization 

In [203]:
from lime.lime_text import LimeTextExplainer
import re

global qid, query, did
tokenizer = lambda doc: re.compile(r"(?u)\b\w\w+\b").findall(doc)
for row in train:
    (qid, did, query, document_text, label) = row
    explainer = LimeTextExplainer(class_names=["irrelevant", "relevant"], split_expression=tokenizer)
    exp = explainer.explain_instance(document_text, predict_proba, num_features=6)
    print(exp.as_list())


Start processing input data for predict stage.

0it [00:00, ?it/s]
1it [00:00, 357.54it/s]
0it [00:00, ?it/s]
31it [00:00, 302.57it/s]
66it [00:00, 314.70it/s]
102it [00:00, 326.42it/s]
136it [00:00, 328.12it/s]
170it [00:00, 330.74it/s]
207it [00:00, 340.10it/s]
248it [00:00, 357.54it/s]
285it [00:00, 360.03it/s]
320it [00:00, 352.00it/s]
364it [00:01, 371.87it/s]
401it [00:01, 362.31it/s]
443it [00:01, 377.37it/s]
481it [00:01, 373.56it/s]
519it [00:01, 362.46it/s]
560it [00:01, 374.50it/s]
603it [00:01, 387.09it/s]
643it [00:01, 389.68it/s]
684it [00:01, 392.84it/s]
724it [00:01, 382.34it/s]
768it [00:02, 394.82it/s]
809it [00:02, 397.27it/s]
849it [00:02, 386.76it/s]
888it [00:02, 369.48it/s]
926it [00:02, 359.59it/s]
963it [00:02, 355.31it/s]
999it [00:02, 347.29it/s]
1034it [00:02, 340.15it/s]
1069it [00:02, 341.02it/s]
1111it [00:03, 359.03it/s]
1151it [00:03, 367.53it/s]
1189it [00:03, 368.77it/s]
1227it [00:03, 361.14it/s]
1264it [00:03, 349.84it/s]
1303it [00:03, 359.34it/s]


[('producer', -0.04499260845471026), ('could', 0.033609363656580946), ('well', 0.03219270320107857), ('that', -0.032107310274637785), ('offer', -0.029095985311904406), ('only', 0.028932719112119233)]


Start processing input data for predict stage.

0it [00:00, ?it/s]
1it [00:00, 455.16it/s]
0it [00:00, ?it/s]
9it [00:00, 86.50it/s]
19it [00:00, 89.52it/s]
30it [00:00, 94.15it/s]
39it [00:00, 90.39it/s]
50it [00:00, 95.22it/s]
59it [00:00, 90.91it/s]
69it [00:00, 92.92it/s]
78it [00:00, 91.75it/s]
88it [00:00, 93.37it/s]
98it [00:01, 90.12it/s]
109it [00:01, 93.99it/s]
119it [00:01, 94.73it/s]
129it [00:01, 95.49it/s]
139it [00:01, 95.18it/s]
149it [00:01, 92.11it/s]
159it [00:01, 88.51it/s]
168it [00:01, 88.09it/s]
181it [00:01, 96.02it/s]
191it [00:02, 95.04it/s]
201it [00:02, 95.30it/s]
211it [00:02, 95.36it/s]
222it [00:02, 96.44it/s]
233it [00:02, 100.06it/s]
244it [00:02, 96.90it/s] 
256it [00:02, 102.75it/s]
267it [00:02, 100.51it/s]
279it [00:02, 105.24it/s]
290it [00:03, 98.97it/s] 
301it [00:03, 92.06it/s]
313it [00:03, 96.70it/s]
323it [00:03, 93.62it/s]
333it [00:03, 92.10it/s]
343it [00:03, 89.38it/s]
353it [00:03, 85.08it/s]
364it [00:03, 90.30it/s]
374it [00:03, 88.22i

[('glare', -0.04167734696674195), ('latest', -0.03816768657701775), ('and', -0.036245562855794354), ('color', 0.03332728953536575), ('subside', 0.030957281445283404), ('when', 0.0304228538274462)]


Start processing input data for predict stage.

0it [00:00, ?it/s]
1it [00:00, 520.32it/s]
0it [00:00, ?it/s]
75it [00:00, 745.20it/s]
158it [00:00, 766.67it/s]
239it [00:00, 778.12it/s]
326it [00:00, 801.85it/s]
413it [00:00, 819.83it/s]
497it [00:00, 823.69it/s]
580it [00:00, 821.74it/s]
667it [00:00, 835.17it/s]
753it [00:00, 839.42it/s]
835it [00:01, 822.34it/s]
916it [00:01, 807.72it/s]
996it [00:01, 781.89it/s]
1081it [00:01, 800.73it/s]
1165it [00:01, 810.17it/s]
1248it [00:01, 815.45it/s]
1333it [00:01, 823.87it/s]
1416it [00:01, 809.80it/s]
1497it [00:01, 809.29it/s]
1582it [00:01, 818.12it/s]
1664it [00:02, 807.89it/s]
1745it [00:02, 798.00it/s]
1825it [00:02, 795.99it/s]
1905it [00:02, 769.57it/s]
1984it [00:02, 772.24it/s]
2065it [00:02, 783.19it/s]
2150it [00:02, 801.94it/s]
2231it [00:02, 794.65it/s]
2318it [00:02, 814.55it/s]
2400it [00:02, 800.14it/s]
2481it [00:03, 796.98it/s]
2569it [00:03, 818.78it/s]
2654it [00:03, 826.90it/s]
2737it [00:03, 807.92it/s]
2819it [00:0

...

[('agno', -0.03613075067880314), ('will', -0.030454284221461292), ('into', 0.025084886355885103), ('to', 0.02301383448046211), ('about', 0.022425418965871795), ('nation', 0.02206508165810989)]


/Users/hamzamanzoor/anaconda/lib/python3.6/site-packages/MatchZoo-2.0.0-py3.6.egg/matchzoo/preprocessor/segment.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  left['length_left'] = left.apply(lambda r: len(r['text_left']), axis=1)
Start processing input data for predict stage.

0it [00:00, ?it/s]
1it [00:00, 406.19it/s]
0it [00:00, ?it/s]
48it [00:00, 475.40it/s]
103it [00:00, 493.91it/s]
152it [00:00, 491.60it/s]
205it [00:00, 500.90it/s]
261it [00:00, 515.56it/s]
322it [00:00, 538.46it/s]
374it [00:00, 529.22it/s]
425it [00:00, 521.69it/s]
479it [00:00, 523.91it/s]
537it [00:01, 539.13it/s]
590it [00:01, 518.29it/s]
645it [00:01, 523.60it/s]
706it [00:01, 545.72it/s]
761it [00:01, 541.58it/s]
816it [00:01, 528.17it/s]
869it [00:01, 511.63it/s]
922it [00:0

[('indefinitely', 0.03399273436291269), ('receive', 0.03087812458887266), ('meet', -0.029174408174068224), ('liability', -0.027955117341882524), ('involved', 0.027404939889662767), ('hold', -0.02698216686936044)]


Start processing input data for predict stage.

0it [00:00, ?it/s]
1it [00:00, 546.70it/s]
0it [00:00, ?it/s]
16it [00:00, 152.93it/s]
37it [00:00, 165.94it/s]
59it [00:00, 179.05it/s]
81it [00:00, 188.87it/s]
104it [00:00, 198.27it/s]
129it [00:00, 209.65it/s]
150it [00:00, 209.51it/s]
173it [00:00, 214.74it/s]
197it [00:00, 220.23it/s]
219it [00:01, 220.07it/s]
241it [00:01, 217.31it/s]
263it [00:01, 217.00it/s]
286it [00:01, 219.63it/s]
311it [00:01, 227.12it/s]
334it [00:01, 225.52it/s]
360it [00:01, 232.43it/s]
384it [00:01, 233.93it/s]
408it [00:01, 232.01it/s]
433it [00:01, 234.08it/s]
458it [00:02, 236.52it/s]
485it [00:02, 244.64it/s]
510it [00:02, 236.48it/s]
535it [00:02, 239.26it/s]
560it [00:02, 230.46it/s]
584it [00:02, 220.68it/s]
607it [00:02, 217.96it/s]
632it [00:02, 226.43it/s]
655it [00:02, 223.71it/s]
679it [00:03, 227.71it/s]
705it [00:03, 236.29it/s]
730it [00:03, 235.72it/s]
755it [00:03, 238.59it/s]
779it [00:03, 238.58it/s]
805it [00:03, 243.63it/s]
830it [00:

KeyboardInterrupt: 

In [19]:
X_predict, _ = generator_predict[0]
pred = dssm_model.predict(X)
for id_left, id_right, pred, _ in zip(X_predict.id_left, X_predict.id_right, pred, range(10)):
    print("{}/{} is predicted as {}".format(id_left, id_right, pred))

Q1733/D1642-14 is predicted as [0.14073879]
Q47/D47-10 is predicted as [0.36516124]
Q2766/D1764-5 is predicted as [-0.02338599]
Q1326/D1268-0 is predicted as [0.01347637]
Q744/D722-12 is predicted as [-0.09433301]
Q952/D920-3 is predicted as [0.7045051]
Q2189/D206-7 is predicted as [0.14717689]
Q1586/D1504-2 is predicted as [-0.33544147]
Q1416/D1349-22 is predicted as [0.08000302]
Q312/D311-2 is predicted as [0.25624424]


#### Model Persistence

You can persist your trained model using `model.save()` and `load_model` function:

In [20]:
dssm_model.save('/tmp/my_dssm_model')
loaded_dssm_model = load_model('/tmp/my_dssm_model')

In [21]:
(loaded_dssm_model.predict(X) == dssm_model.predict(X)).all()

True

## Reference

[Huang et al. 2013] Po-Sen Huang, Xiaodong He, Jianfeng Gao, Li Deng, Alex Acero, and Larry Heck. 2013. Learning deep structured semantic models for web search using clickthrough data. In Proc. CIKM. ACM, 2333–2338.